In [2]:
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
import random
import seaborn as sns
import scipy
from scipy.stats import pearsonr
import sklearn
from sklearn import datasets, linear_model
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
import math
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.neural_network import MLPRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso
from sklearn.preprocessing import minmax_scale
from numpy import std, mean
import statistics
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SequentialFeatureSelector


#Import File
global all_df
all_df=pd.read_csv('TrainDataset2023.csv', index_col=False) #Read from File
all_df.drop('ID', axis=1, inplace=True) # Drop ID - not needed for training

#Impute Missing Values
imputer = SimpleImputer(missing_values = 999, strategy="median") # Test Other Methods
SimpleImputer(missing_values = 999)
s = 0
for i in all_df:
    imputer.fit(all_df)
    array = np.array(all_df[i])
    all_df[i] = imputer.fit_transform(array.reshape(-1, 1))       
    
#Normalise Values

#Min Max 
colno = 0
for col in all_df:
    if col == 'RelapseFreeSurvival (outcome)':
        continue
    if colno >= 12: # ONLY NORMALISE MRI SCAN DATA
        colmean = np.median(all_df[col])
        colstd = np.std(all_df[col])
        upper = colmean + (3*colstd)
        lower = colmean - (3*colstd) # USING MIN ALSO SEEMS TO SKEW DATA
        #Comment out to cancel
        #all_df[col] = minmax_scale(all_df[col], feature_range=(lower,upper)) #Minimal Change - Downscales severity of Mean Squared Error
    colno+=1
    
print('Data Normilisation Complete')

Data Normilisation Complete


In [ ]:
print('SVM Regression Start \n')
y = all_df['RelapseFreeSurvival (outcome)']
x = all_df.drop('RelapseFreeSurvival (outcome)', axis=1)
train_X, validate_X, train_y, validate_y = train_test_split(x, y, test_size=0.15, shuffle = False)
regressor = SVR(kernel = 'sigmoid')

def FFS(trainx, trainy, validationx, validationy, maxscore):
    F = trainx.columns.tolist()
    #F.remove('RelapseFreeSurvival (outcome)')
    X = []
    B = maxscore
    while X != F:
        Y = []
        remfeatures = list(set(F)-set(Y))
        for i in remfeatures:
            temp = Y+[i]
            regressor.fit(trainx[temp],trainy)
            y_pred = regressor.predict(validationx[temp])
            mse = mean_absolute_error(validationy, y_pred)
            if mse < B:#-(B/80): #Fine Tune the Optimal Increase for a new feature to be worthwhile
                B = mean_absolute_error(validationy, y_pred)
                Y = temp
                #print("Features Updated: "+str(Y))
                #print("Best Score Updated: "+str(B))
        if X != [] and mean_absolute_error(validationy, y_pred) < B:
            #print("Best X Features = "+str(X))
            #print("Best Features Accuracy = "+str(B))
            break
        else:
            X = Y
            break
    print("Best Features = "+str(X))
    print('Number of Features Used = '+str(len(X)))
    #y_pred = lin_reg.predict(validationx[X])
    #print("Feature Mean Squared Error = "+str(mean_squared_error(validationy, y_pred)))
    print("")
    return X
        
y = all_df['RelapseFreeSurvival (outcome)']
X = FFS(train_X, train_y, validate_X, validate_y, 1000) #Test FFS
scaler = StandardScaler()
Xs = scaler.fit_transform(train_X[X])
mse_total = 0
mse2_total = 0
score_total = 0
kf = KFold(n_splits=5)
for train, test in kf.split(Xs, train_y):
    regressor.fit(Xs[train],y[train])
    regression_score = regressor.score(Xs[test], y[test])
    y_pred = regressor.predict(Xs[test])
    mse = mean_absolute_error(y[test], y_pred)
    y_pred2 = regressor.predict(Xs[train])
    mse2 = mean_absolute_error(y[train], y_pred2)
    print('TEST')
    print(mse)
    print('TRAIN')
    print(mse2)
    score_total += regression_score
    mse_total += mse
    mse2_total += mse2
    print('')

print('Validation Set: ')
print('The Avg. Accuracy of Linear Regression is {:03.2f}'.format(score_total/5))
print('The Avg. Mean Absolute Error for Linear Regression Test is {:03.4f}'.format(mse_total/5))
print('The Avg. Mean Absolute Error for Linear Regression Train is {:03.4f}'.format(mse2_total/5))
print("")

In [ ]:
print('SVM Regression Start \n')
y = all_df['RelapseFreeSurvival (outcome)']
x = all_df.drop('RelapseFreeSurvival (outcome)', axis=1)
train_X, validate_X, train_y, validate_y = train_test_split(x, y, test_size=0.15, shuffle = False)
regressor = SVR(kernel = 'sigmoid')

featureno = 1
score = 10000
y = all_df['RelapseFreeSurvival (outcome)']
x = all_df.drop('RelapseFreeSurvival (outcome)', axis=1)
while featureno < 50:
    featuretest = []
    sfs = SequentialFeatureSelector(regressor, n_features_to_select=featureno)
    sfs.fit(x,y)
    feats = sfs.get_feature_names_out() 
    for i in feats:
        featuretest.append(i)
    x2 = train_X[featuretest] 
    regressor.fit(x2,train_y)
    y_pred = regressor.predict(validate_X[featuretest])
    mse = mean_absolute_error(validate_y, y_pred)
    if mse < score:#-(score/20):
        featureno+=1
        score = mse
        continue
    else:
        break
    
print("SFS Best No. of Features = "+str(featureno))
    
sfs = SequentialFeatureSelector(regressor, n_features_to_select=featureno)
sfs.fit(x,y)
feats = sfs.get_feature_names_out() 
print(feats)
feature = []
for i in feats:
    feature.append(i)

y = all_df['RelapseFreeSurvival (outcome)']
x = train_X[feature]
Xs = scaler.fit_transform(x) 
mse_total = 0
mse2_total = 0
score_total = 0
kf = KFold(n_splits=5)
for train, test in kf.split(Xs, train_y):
    regressor.fit(Xs[train],y[train])
    regression_score = regressor.score(Xs[test], y[test])
    y_pred = regressor.predict(Xs[test])
    mse = mean_absolute_error(y[test], y_pred)
    y_pred2 = regressor.predict(Xs[train])
    mse2 = mean_absolute_error(y[train], y_pred2)
    print('TEST')
    print(mse)
    print('TRAIN')
    print(mse2)
    score_total += regression_score
    mse_total += mse
    mse2_total += mse2
    print('')
    
print('Validation Set: ')
print('The Avg. Accuracy of Linear Regression is {:03.2f}'.format(score_total/5))
print('The Avg. Mean Absolute Error for Linear Regression Test is {:03.4f}'.format(mse_total/5))
print('The Avg. Mean Absolute Error for Linear Regression Train is {:03.4f}'.format(mse2_total/5))
print("")

In [ ]:

print('\nDecision Tree Regression Start \n')

x = all_df.drop('RelapseFreeSurvival (outcome)', axis=1)
y = all_df['RelapseFreeSurvival (outcome)']
scaler = StandardScaler()
Xs = scaler.fit_transform(x)
train_X, validate_X, train_y, validate_y = train_test_split(x, y, test_size=0.15, shuffle = False)
def RFDepthSelection(trainx, trainy, testx, testy):
    feature_limit = 1
    mse_score_feature = 1000
    mse_score_depth = 1000
    while feature_limit <= 114:
        print('Feature Loop '+str(feature_limit))
        mse_score_depth = 1000
        depth_limit = 1
        while depth_limit <= 114:
            print('Depth Loop '+str(depth_limit))
            rfregressor = RandomForestRegressor(n_estimators=1000, random_state = 0, criterion = 'absolute_error',
                                                max_features = feature_limit, max_depth = depth_limit)
            rfregressor.fit(trainx, trainy)
            y_pred = rfregressor.predict(testx)
            mse = mean_absolute_error(testy, y_pred)
            if mse < mse_score_depth:# - (mse_score/20):
                mse_score_depth = mse
                depth_limit+=1
                continue
            else:
                break
        print('CHECK MSE')
        print('MSE = '+str(mse))
        print('MSE_SCORE Feature = '+str(mse_score_feature))
        print('MSE SCORE Depth = '+str(mse_score_depth))
        if mse_score_depth <= mse_score_feature:
            mse_score_feature = mse_score_depth
            feature_limit += 1
            continue
        else:
            return [feature_limit, depth_limit]

outcome = RFDepthSelection(train_X, train_y, validate_X, validate_y)
print('Best Feature = '+str(outcome[0]))
print("Best Depth = "+str(outcome[1]))

#K-fold
scaler = StandardScaler()
Xs = scaler.fit_transform(train_X)
mse_total = 0
mse2_total = 0
score_total = 0
kf = KFold(n_splits=5)
tree_clf = DecisionTreeRegressor(max_depth=outcome[1], max_features=outcome[0])
for train, test in kf.split(Xs, train_y):
    tree_clf.fit(Xs[train],y[train])
    regression_score = tree_clf.score(Xs[test], y[test])
    y_pred = tree_clf.predict(Xs[test])
    mse = mean_absolute_error(y[test], y_pred)
    y_pred2 = tree_clf.predict(Xs[train])
    mse2 = mean_absolute_error(y[train], y_pred2)
    print('TEST')
    print(mse)
    print('TRAIN')
    print(mse2)
    print('')
    score_total += regression_score
    mse_total += mse
    mse2_total += mse2

print('The Avg. Accuracy of Decision Tree Regression is {:03.2f}'.format(score_total/5))
print('The Avg. Mean Square Error for Decision Tree Regression Test is {:03.4f}'.format(mse_total/5))
print('The Avg. Mean Square Error for Decision Tree Regression Training is {:03.4f}'.format(mse2_total/5))
print('\nDecision Tree Regression End \n')
# Decision Tree Regression End

In [4]:
print('\nNeural Network Regression Start \n')

x = all_df.drop('RelapseFreeSurvival (outcome)', axis=1)
y = all_df['RelapseFreeSurvival (outcome)']
train_X, validate_X, train_y, validate_y = train_test_split(x, y, test_size=0.15, shuffle = False)

def hyperparameter(trainx, trainy, validatex, validatey):
    hyperparameterlimit = 1
    bestlimit = 1
    mse_score = 1000
    while hyperparameterlimit <= 200:
        print('Hyperparameter loop no. '+str(hyperparameterlimit))
        mlp_clf = MLPRegressor(random_state=1, max_iter=400, 
                               hidden_layer_sizes = hyperparameterlimit).fit(trainx, trainy)
        y_pred = mlp_clf.predict(validatex)
        mse = mean_absolute_error(validatey, y_pred)
        print(hyperparameterlimit)
        print(mse)
        if mse < mse_score:# - (mse_score/20):
            mse_score = mse
            bestlimit = hyperparameterlimit
        hyperparameterlimit+=1
    return hyperparameterlimit
# Hidden Layer size = minimal difference to test - only effects training



scaler = StandardScaler()
Xs = scaler.fit_transform(x)
#hyperparameterval = hyperparameter(train_X, train_y, validate_X, validate_y) 
#K-fold
mse_total = 0
mse2_total = 0
score_total = 0
kf = KFold(n_splits=5)
for train, test in kf.split(Xs, y):
    mlp_clf = MLPRegressor(random_state=1, max_iter=8,
                          activation = 'relu', solver = 'lbfgs', learning_rate = 'adaptive',
                          hidden_layer_sizes = 5).fit(Xs[train], y[train])
    regression_score = mlp_clf.score(Xs[test], y[test])
    y_pred = mlp_clf.predict(Xs[test])
    mse = mean_absolute_error(y[test], y_pred)
    y_pred2 = mlp_clf.predict(Xs[train])
    mse2 = mean_absolute_error(y[train], y_pred2)
    print('TEST')
    print(mse)
    print('TRAIN')
    print(mse2)
    print('')
    print(mlp_clf.n_layers_)
    score_total += regression_score
    mse_total += mse
    mse2_total += mse2

print('The Avg. Accuracy of Neural Network Regression is {:03.2f}'.format(score_total/5))
print('The Avg. Mean Square Error for Neural Network Regression Test is {:03.4f}'.format(mse_total/5))
print('The Avg. Mean Square Error for Neural Network Regression Training is {:03.4f}'.format(mse2_total/5))
print('\nNeural Network Regression End \n')
# Neural Network Regression End


Neural Network Regression Start 

TEST
41.67398357660278
TRAIN
14.675967864572465

3
TEST
20.20433682149335
TRAIN
19.70702783185494

3
TEST
15.746851941750743
TRAIN
19.996453965601063

3
TEST
22.79139379345222
TRAIN
19.369565285253636

3
TEST
19.343575267124997
TRAIN
19.568827309127137

3
The Avg. Accuracy of Neural Network Regression is -1.11
The Avg. Mean Square Error for Neural Network Regression Test is 23.9520
The Avg. Mean Square Error for Neural Network Regression Training is 18.6636

Neural Network Regression End 



In [ ]:
print('\nRandom Forest Regression Start \n')

x = all_df.drop('RelapseFreeSurvival (outcome)', axis=1)
y = all_df['RelapseFreeSurvival (outcome)']
train_X, validate_X, train_y, validate_y = train_test_split(x, y, test_size=0.15, shuffle = False)
#Find best feature to accuracy ratio - random forest has built in feature selection via feature ranking
# return Highest rated features
def RFDepthSelection(trainx, trainy, testx, testy):
    feature_limit = 1
    mse_score_feature = 1000
    mse_score_depth = 1000
    while feature_limit <= 114:
        #rint('Feature Loop '+str(feature_limit))
        mse_score_depth = 1000
        depth_limit = 1
        while depth_limit <= 114:
            #rint('Depth Loop '+str(depth_limit))
            rfregressor = RandomForestRegressor(n_estimators=1000, random_state = 0, criterion = 'absolute_error',
                                                max_features = feature_limit, max_depth = depth_limit)
            rfregressor.fit(trainx, trainy)
            y_pred = rfregressor.predict(testx)
            mse = mean_absolute_error(testy, y_pred)
            if mse < mse_score_depth:# - (mse_score/20):
                mse_score_depth = mse
                depth_limit+=1
                continue
            else:
                break
        #rint('CHECK MSE')
        #rint('MSE = '+str(mse))
        #rint('MSE_SCORE Feature = '+str(mse_score_feature))
        #rint('MSE SCORE Depth = '+str(mse_score_depth))
        if mse_score_depth <= mse_score_feature:
            mse_score_feature = mse_score_depth
            feature_limit += 1
            continue
        else:
            return [feature_limit, depth_limit]

validateoutcome = RFDepthSelection(train_X, train_y, validate_X, validate_y)
feature_limit = validateoutcome[0]
depth_limit = validateoutcome[1]
rfregressor = RandomForestRegressor(n_estimators=1000, random_state = 0, criterion = 'absolute_error',
                                            max_features = feature_limit, max_depth = depth_limit)
print("Feature Limit = "+str(feature_limit))
print("Depth Limit = "+str(depth_limit))

#Make x the validated feature
#rfregressor = RandomForestRegressor(n_estimators=100, random_state = 0, max_features = feature_limit) #Default Measure = MSE
#K-fold
scaler = StandardScaler()
Xs = scaler.fit_transform(train_X)
mse_total = 0
mse2_total = 0
score_total = 0
kf = KFold(n_splits=5)
for train, test in kf.split(Xs, train_y):
    #featureno = RFDepthSelection(Xs[train], y[train], Xs[test], y[test]) #Use Nested Kfold for this?
    #print(featureno)
    #rfregressor = RandomForestRegressor(n_estimators=1000, random_state = 0, max_features = featureno)
    rfregressor.fit(Xs[train], y[train])
    y_pred = rfregressor.predict(Xs[test])
    mse = mean_absolute_error(y[test], y_pred)
    y_pred2 = rfregressor.predict(Xs[train])
    mse2 = mean_absolute_error(y[train], y_pred2)
    print('TEST')
    print(mse)
    print('TRAIN')
    print(mse2)
    print('')
    mse_total += mse
    mse2_total += mse2

#print("Best Number of Features = "+str(feature_limit))
print('The Avg. Mean Square Error for Random Forest Regression Test is {:03.4f}'.format(mse_total/5))
print('The Avg. Mean Square Error for Random Forest Regression Train is {:03.4f}'.format(mse2_total/5))
print('\nRandom Forest Regression End \n')

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
x = all_df.drop('RelapseFreeSurvival (outcome)', axis=1)
y = all_df['RelapseFreeSurvival (outcome)']
train_X, validate_X, train_y, validate_y = train_test_split(x, y, test_size=0.15, shuffle = False)
#Find best feature to accuracy ratio - random forest has built in feature selection via feature ranking
# return Highest rated features
def RFDepthSelection(trainx, trainy, testx, testy):
    feature_limit = 1
    mse_score_feature = 1000
    mse_score_depth = 1000
    while feature_limit <= 114:
        print('Feature Loop '+str(feature_limit))
        mse_score_depth = 1000
        depth_limit = 1
        while depth_limit <= 114:
            print('Depth Loop '+str(depth_limit))
            rfregressor = ExtraTreesRegressor(n_estimators=1000, random_state = 0, criterion = 'absolute_error',
                                                max_features = feature_limit, max_depth = depth_limit)
            rfregressor.fit(trainx, trainy)
            y_pred = rfregressor.predict(testx)
            mse = mean_absolute_error(testy, y_pred)
            if mse < mse_score_depth:# - (mse_score/20):
                mse_score_depth = mse
                depth_limit+=1
                continue
            else:
                break
        print('CHECK MSE')
        print('MSE = '+str(mse))
        print('MSE_SCORE Feature = '+str(mse_score_feature))
        print('MSE SCORE Depth = '+str(mse_score_depth))
        if mse_score_depth <= mse_score_feature:
            mse_score_feature = mse_score_depth
            feature_limit += 1
            continue
        else:
            return [feature_limit, depth_limit]

validateoutcome = RFDepthSelection(train_X, train_y, validate_X, validate_y)
feature_limit = validateoutcome[0]
depth_limit = validateoutcome[1]
rfregressor = ExtraTreesRegressor(n_estimators=1000, random_state = 0, criterion = 'absolute_error',
                                            max_features = feature_limit, max_depth = depth_limit)
print("Feature Limit = "+str(feature_limit))
print("Depth Limit = "+str(depth_limit))

#Make x the validated feature
#rfregressor = RandomForestRegressor(n_estimators=100, random_state = 0, max_features = feature_limit) #Default Measure = MSE
#K-fold
scaler = StandardScaler()
Xs = scaler.fit_transform(train_X)
mse_total = 0
mse2_total = 0
score_total = 0
kf = KFold(n_splits=5)
for train, test in kf.split(Xs, train_y):
    #featureno = RFDepthSelection(Xs[train], y[train], Xs[test], y[test]) #Use Nested Kfold for this?
    #print(featureno)
    #rfregressor = RandomForestRegressor(n_estimators=1000, random_state = 0, max_features = featureno)
    rfregressor.fit(Xs[train], y[train])
    y_pred = rfregressor.predict(Xs[test])
    mse = mean_absolute_error(y[test], y_pred)
    y_pred2 = rfregressor.predict(Xs[train])
    mse2 = mean_absolute_error(y[train], y_pred2)
    print('TEST')
    print(mse)
    print('TRAIN')
    print(mse2)
    print('')
    mse_total += mse
    mse2_total += mse2

#print("Best Number of Features = "+str(feature_limit))
print('The Avg. Mean Square Error for Random Forest Regression Test is {:03.4f}'.format(mse_total/5))
print('The Avg. Mean Square Error for Random Forest Regression Train is {:03.4f}'.format(mse2_total/5))
print('\nRandom Forest Regression End \n')

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor
x = all_df.drop('RelapseFreeSurvival (outcome)', axis=1)
y = all_df['RelapseFreeSurvival (outcome)']
train_X, validate_X, train_y, validate_y = train_test_split(x, y, test_size=0.15, shuffle = False)
#Find best feature to accuracy ratio - random forest has built in feature selection via feature ranking
# return Highest rated features
def RFDepthSelection(trainx, trainy, testx, testy):
    depth_limit = 1
    mse_score_feature = 1000
    mse_score_depth = 1000
    while depth_limit <= 114:
        print('Depth Loop '+str(depth_limit))
        rfregressor = HistGradientBoostingRegressor(max_iter = 10, max_depth = depth_limit)
        rfregressor.fit(trainx, trainy)
        y_pred = rfregressor.predict(testx)
        mse = mean_absolute_error(testy, y_pred)
        if mse < mse_score_depth:# - (mse_score/20):
            mse_score_depth = mse
            depth_limit+=1
            continue
        else:
            return depth_limit

validateoutcome = RFDepthSelection(train_X, train_y, validate_X, validate_y)
rfregressor = HistGradientBoostingRegressor(max_iter = 10, max_depth = validateoutcome)
print("Depth Limit = "+str(validateoutcome))

#Make x the validated feature
#rfregressor = RandomForestRegressor(n_estimators=100, random_state = 0, max_features = feature_limit) #Default Measure = MSE
#K-fold
scaler = StandardScaler()
Xs = scaler.fit_transform(train_X)
mse_total = 0
mse2_total = 0
score_total = 0
kf = KFold(n_splits=5)
for train, test in kf.split(Xs, train_y):
    #featureno = RFDepthSelection(Xs[train], y[train], Xs[test], y[test]) #Use Nested Kfold for this?
    #print(featureno)
    #rfregressor = RandomForestRegressor(n_estimators=1000, random_state = 0, max_features = featureno)
    rfregressor.fit(Xs[train], y[train])
    y_pred = rfregressor.predict(Xs[test])
    mse = mean_absolute_error(y[test], y_pred)
    y_pred2 = rfregressor.predict(Xs[train])
    mse2 = mean_absolute_error(y[train], y_pred2)
    print('TEST')
    print(mse)
    print('TRAIN')
    print(mse2)
    print('')
    mse_total += mse
    mse2_total += mse2

#print("Best Number of Features = "+str(feature_limit))
print('The Avg. Mean Square Error for Random Forest Regression Test is {:03.4f}'.format(mse_total/5))
print('The Avg. Mean Square Error for Random Forest Regression Train is {:03.4f}'.format(mse2_total/5))
print('\nRandom Forest Regression End \n')

In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
x = all_df.drop('RelapseFreeSurvival (outcome)', axis=1)
y = all_df['RelapseFreeSurvival (outcome)']
train_X, validate_X, train_y, validate_y = train_test_split(x, y, test_size=0.15, shuffle = False)
reg = make_pipeline(StandardScaler(), SGDRegressor(max_iter=1000, tol=1e-3, shuffle = False, penalty='elasticnet'))
reg.fit(x,y)
Xs = scaler.fit_transform(x)
mse_total = 0
mse2_total = 0
score_total = 0
kf = KFold(n_splits=5)
for train, test in kf.split(Xs, y):
    #featureno = RFDepthSelection(Xs[train], y[train], Xs[test], y[test]) #Use Nested Kfold for this?
    #print(featureno)
    #rfregressor = RandomForestRegressor(n_estimators=1000, random_state = 0, max_features = featureno)
    reg.fit(Xs[train], y[train])
    y_pred = reg.predict(Xs[test])
    mse = mean_absolute_error(y[test], y_pred)
    y_pred2 = reg.predict(Xs[train])
    mse2 = mean_absolute_error(y[train], y_pred2)
    print('TEST')
    print(mse)
    print('TRAIN')
    print(mse2)
    print('')
    mse_total += mse
    mse2_total += mse2

#print("Best Number of Features = "+str(feature_limit))
print('The Avg. Mean Square Error for SGD Test is {:03.4f}'.format(mse_total/5))
print('The Avg. Mean Square Error for SGD Train is {:03.4f}'.format(mse2_total/5))
print('\nRandom Forest Regression End \n')

In [ ]:
from sklearn.linear_model import HuberRegressor
from sklearn.datasets import make_regression
x = all_df.drop('RelapseFreeSurvival (outcome)', axis=1)
y = all_df['RelapseFreeSurvival (outcome)']
train_X, validate_X, train_y, validate_y = train_test_split(x, y, test_size=0.15, shuffle = False)

huber=HuberRegressor()
Xs = scaler.fit_transform(x)
mse_total = 0
mse2_total = 0
score_total = 0
kf = KFold(n_splits=5)
for train, test in kf.split(Xs, y):
    #featureno = RFDepthSelection(Xs[train], y[train], Xs[test], y[test]) #Use Nested Kfold for this?
    #print(featureno)
    #rfregressor = RandomForestRegressor(n_estimators=1000, random_state = 0, max_features = featureno)
    huber.fit(Xs[train], y[train])
    y_pred = huber.predict(Xs[test])
    mse = mean_absolute_error(y[test], y_pred)
    y_pred2 = huber.predict(Xs[train])
    mse2 = mean_absolute_error(y[train], y_pred2)
    print('TEST')
    print(mse)
    print('TRAIN')
    print(mse2)
    print('')
    mse_total += mse
    mse2_total += mse2
#print("Best Number of Features = "+str(feature_limit))
print('The Avg. Mean Square Error for Huber Test is {:03.4f}'.format(mse_total/5))
print('The Avg. Mean Square Error for Huber Train is {:03.4f}'.format(mse2_total/5))
print('\nRandom Forest Regression End \n')

In [ ]:
from sklearn.linear_model import RANSACRegressor
x = all_df.drop('RelapseFreeSurvival (outcome)', axis=1)
y = all_df['RelapseFreeSurvival (outcome)']
train_X, validate_X, train_y, validate_y = train_test_split(x, y, test_size=0.15, shuffle = False)

reg=RANSACRegressor()
Xs = scaler.fit_transform(x)
mse_total = 0
mse2_total = 0
score_total = 0
kf = KFold(n_splits=5)
for train, test in kf.split(Xs, y):
    #featureno = RFDepthSelection(Xs[train], y[train], Xs[test], y[test]) #Use Nested Kfold for this?
    #print(featureno)
    #rfregressor = RandomForestRegressor(n_estimators=1000, random_state = 0, max_features = featureno)
    reg.fit(Xs[train], y[train])
    y_pred = reg.predict(Xs[test])
    mse = mean_absolute_error(y[test], y_pred)
    y_pred2 = reg.predict(Xs[train])
    mse2 = mean_absolute_error(y[train], y_pred2)
    print('TEST')
    print(mse)
    print('TRAIN')
    print(mse2)
    print('')
    mse_total += mse
    mse2_total += mse2
#print("Best Number of Features = "+str(feature_limit))
print('The Avg. Mean Square Error for RANSAC Test is {:03.4f}'.format(mse_total/5))
print('The Avg. Mean Square Error for RANSAC Train is {:03.4f}'.format(mse2_total/5))
print('\nRandom Forest Regression End \n')

In [ ]:
from sklearn.linear_model import TheilSenRegressor
x = all_df.drop('RelapseFreeSurvival (outcome)', axis=1)
y = all_df['RelapseFreeSurvival (outcome)']
train_X, validate_X, train_y, validate_y = train_test_split(x, y, test_size=0.15, shuffle = False)

reg=TheilSenRegressor()
Xs = scaler.fit_transform(x)
mse_total = 0
mse2_total = 0
score_total = 0
kf = KFold(n_splits=5)
for train, test in kf.split(Xs, y):
    #featureno = RFDepthSelection(Xs[train], y[train], Xs[test], y[test]) #Use Nested Kfold for this?
    #print(featureno)
    #rfregressor = RandomForestRegressor(n_estimators=1000, random_state = 0, max_features = featureno)
    reg.fit(Xs[train], y[train])
    y_pred = reg.predict(Xs[test])
    mse = mean_absolute_error(y[test], y_pred)
    y_pred2 = reg.predict(Xs[train])
    mse2 = mean_absolute_error(y[train], y_pred2)
    print('TEST')
    print(mse)
    print('TRAIN')
    print(mse2)
    print('')
    mse_total += mse
    mse2_total += mse2
#print("Best Number of Features = "+str(feature_limit))
print('The Avg. Mean Square Error for Theil Test is {:03.4f}'.format(mse_total/5))
print('The Avg. Mean Square Error for Theil Train is {:03.4f}'.format(mse2_total/5))

In [ ]:
from sklearn.linear_model import Ridge
x = all_df.drop('RelapseFreeSurvival (outcome)', axis=1)
y = all_df['RelapseFreeSurvival (outcome)']
train_X, validate_X, train_y, validate_y = train_test_split(x, y, test_size=0.15, shuffle = False)

reg=Ridge(alpha=1.0)
Xs = scaler.fit_transform(x)
mse_total = 0
mse2_total = 0
score_total = 0
kf = KFold(n_splits=5)
for train, test in kf.split(Xs, y):
    #featureno = RFDepthSelection(Xs[train], y[train], Xs[test], y[test]) #Use Nested Kfold for this?
    #print(featureno)
    #rfregressor = RandomForestRegressor(n_estimators=1000, random_state = 0, max_features = featureno)
    reg.fit(Xs[train], y[train])
    y_pred = reg.predict(Xs[test])
    mse = mean_absolute_error(y[test], y_pred)
    y_pred2 = reg.predict(Xs[train])
    mse2 = mean_absolute_error(y[train], y_pred2)
    print('TEST')
    print(mse)
    print('TRAIN')
    print(mse2)
    print('')
    mse_total += mse
    mse2_total += mse2
#print("Best Number of Features = "+str(feature_limit))
print('The Avg. Mean Square Error for Ridge Test is {:03.4f}'.format(mse_total/5))
print('The Avg. Mean Square Error for Ridge Train is {:03.4f}'.format(mse2_total/5))